# Importing

In [1]:
#Importing Packages
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#games_metadata = pd.read_json("/content/drive/MyDrive/BT4222/Data/games_metadata.json", lines=True)
#games_data = pd.read_csv("/content/drive/MyDrive/BT4222/Data/games.csv")
games_metadata = pd.read_json("games_metadata.json", lines=True)
games_data = pd.read_csv("games_cleaned.csv")
recommendations_data = pd.read_csv("recommendations_with_score.csv")
users_data = pd.read_csv("sample_user_data.csv")
cluster_data = pd.read_csv("clustering.csv")

# Data Processing

In [3]:
#Games Metadata
games_metadata.head(5)

,app_id,description,tags
0,13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."
1,22364,,[Action]
2,113020,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
3,226560,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P..."
4,249050,Dungeon of the Endless is a Rogue-Like Dungeon...,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."


In [4]:
#Games Dataset
games_data.head(5)

,app_id,title,date_release,win,mac,linux,rating_encoded,positive_ratio_log,user_reviews_log,price_final_log,...,Hand-drawn,Foreign,Narration,Job Simulator,Comedy,Linear,Shop Keeper,Audio Production,Submarine,Hack and Slash
0,13500,Prince of Persia: Warrior Within™,2008-11-21,1,0,0,7.0,4.442651,7.695758,2.396986,...,0,0,0,0,0,0,0,0,0,1
1,22364,BRINK: Agents of Change,2011-08-03,1,0,0,6.0,4.454347,3.044522,1.383791,...,0,0,0,0,0,0,0,0,0,0
2,113020,Monaco: What's Yours Is Mine,2013-04-24,1,1,1,7.0,4.532599,8.222016,2.771964,...,0,0,0,0,0,0,0,0,0,0
3,226560,Escape Dead Island,2014-11-18,1,0,0,4.0,4.127134,6.771936,2.771964,...,0,0,0,0,0,0,0,0,0,1
4,249050,Dungeon of the ENDLESS™,2014-10-27,1,1,0,7.0,4.488636,9.080687,2.564180,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#Truncate Game Dataset
games_data = games_data[['app_id','title', 'rating_encoded', 'positive_ratio_log']]
games_data.head(5)

,app_id,title,rating_encoded,positive_ratio_log
0,13500,Prince of Persia: Warrior Within™,7.0,4.442651
1,22364,BRINK: Agents of Change,6.0,4.454347
2,113020,Monaco: What's Yours Is Mine,7.0,4.532599
3,226560,Escape Dead Island,4.0,4.127134
4,249050,Dungeon of the ENDLESS™,7.0,4.488636


In [6]:
#Users Dataset
print(len(users_data))
users_data.head(5)

30000


,user_id,products,reviews,average_recommendation_credibility,reviews_scaled,products_scaled,average_recommendation_credibility_scaled,user_credibility,user_credibility_log
0,947,216.0,12.0,0.000000,0.004924,0.007260,0.000000,0.004381,0.004372
1,1107,242.0,9.0,0.000000,0.003693,0.008134,0.000000,0.004361,0.004352
2,1345,149.0,9.0,0.000000,0.003693,0.005008,0.000000,0.003111,0.003106
3,1405,580.0,23.0,0.059939,0.009438,0.019494,0.108541,0.043191,0.042285
4,1634,168.0,23.0,0.000000,0.009438,0.005647,0.000000,0.005090,0.005077


In [7]:
#Recommendations Dataset
recommendations_data.head(5)

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,helpful_score,hours_log,score,date_factor,recommendation_credibility,recommendation_credibility_normalized,recommendation_credibility_normalized_log
0,1419170,0,0,2021-08-13,1,0.4,3305939,253,1.0,0.336472,2,0.333333,0.003589,0.00650,0.006479
1,239030,2,0,2019-01-08,1,34.4,5755947,268,2.0,3.566712,4,0.200000,0.045654,0.08267,0.079430
2,311210,0,0,2021-08-03,1,104.3,2323347,510,1.0,4.656813,2,0.333333,0.049673,0.08995,0.086132
3,570,4,0,2013-12-16,1,893.0,4572014,868,3.0,6.795706,10,0.090909,0.059308,0.10740,0.102015
4,1284210,0,0,2022-09-04,1,0.4,7273338,1172,1.0,0.336472,1,0.500000,0.005384,0.00975,0.009703


In [8]:
#Filter Recommendations only with users in the user dataset
recommendations_data = recommendations_data[recommendations_data['user_id'].isin(users_data['user_id'])]

# Sort by user_id
recommendations_data = recommendations_data.sort_values(by='user_id', ascending=True)
recommendations_data = recommendations_data.reset_index(drop=True)
print(len(recommendations_data)) # Number of recommendations in dataset
recommendations_data.head(5)

531429


,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,helpful_score,hours_log,score,date_factor,recommendation_credibility,recommendation_credibility_normalized,recommendation_credibility_normalized_log
0,1066900,0,0,2021-11-26,1,1.1,947,36579228,1.0,0.741937,2,0.333333,0.007914,0.01433,0.014228
1,292030,0,0,2021-02-15,1,79.2,947,2543284,1.0,4.384524,2,0.333333,0.046768,0.08469,0.081294
2,964800,12,4,2022-09-27,-1,11.2,947,16821258,5.0,2.501436,1,0.500000,0.200115,0.36239,0.309241
3,1036890,0,2,2022-03-04,1,6.2,947,8967417,1.0,1.974081,1,0.500000,0.031585,0.05720,0.055624
4,312990,5,0,2017-07-20,1,1.0,947,37243937,4.0,0.693147,6,0.142857,0.012675,0.02295,0.022691


In [9]:
#Merge Games Data and Games Metadata
games_df = pd.merge(games_data, games_metadata, how='inner', on='app_id')
print(len(games_df)) # Number of games in games dataset

#Remove games that have no descriptions or tags
games_df = games_df[~((games_df['description'] == "") | (games_df['tags'].apply(len) == 0))]

#Sort by app_id
games_df = games_df.sort_values(by=['app_id'])
games_df = games_df.reset_index(drop=True)
print(len(games_df)) # Number of games with either a description or tag
games_df.head()

50872
40484


,app_id,title,rating_encoded,positive_ratio_log,description,tags
0,10,Counter-Strike,8.0,4.584967,Play the world's number 1 online action game. ...,"[Action, FPS, Multiplayer, Shooter, Classic, T..."
1,20,Team Fortress Classic,7.0,4.454347,One of the most popular online action games of...,"[Action, FPS, Multiplayer, Classic, Hero Shoot..."
2,30,Day of Defeat,7.0,4.477337,Enlist in an intense brand of Axis vs. Allied ...,"[FPS, World War II, Multiplayer, Shooter, Acti..."
3,40,Deathmatch Classic,5.0,4.382027,Enjoy fast-paced multiplayer gaming with Death...,"[Action, FPS, Classic, Multiplayer, Shooter, F..."
4,50,Half-Life: Opposing Force,8.0,4.564348,Return to the Black Mesa Research Facility as ...,"[FPS, Action, Classic, Sci-fi, Singleplayer, S..."


In [10]:
#Truncating recommendation data
recommendations_short_data = recommendations_data[['app_id','user_id','is_recommended']]
recommendations_short_data =  recommendations_short_data[recommendations_short_data['app_id'].isin(games_df['app_id'])]
recommendations_short_data = recommendations_short_data.sort_values(by='user_id', ascending=True)
recommendations_short_data = recommendations_short_data.reset_index(drop=True)
recommendations_short_data.head(5)

,app_id,user_id,is_recommended
0,1066900,947,1
1,964800,947,-1
2,1036890,947,1
3,312990,947,1
4,1511780,947,1


In [11]:
#Clustering data
cluster_data = cluster_data.sort_values(by='user_id', ascending=True)
cluster_data =  cluster_data[cluster_data['app_id'].isin(games_df['app_id'])]
cluster_data.head(10)

,app_id,helpful,funny,is_recommended,hours,user_id,review_id,helpful_score,hours_log,score,date_factor,recommendation_credibility,recommendation_credibility_normalized,recommendation_credibility_normalized_log,cluster_label
457798,1066900,0,0,1,1.1,947,36579228,1.0,0.741937,2,0.333333,0.007914,0.01433,0.014228,[2]
174474,964800,12,4,-1,11.2,947,16821258,5.0,2.501436,1,0.500000,0.200115,0.36239,0.309241,[2]
79193,1036890,0,2,1,6.2,947,8967417,1.0,1.974081,1,0.500000,0.031585,0.05720,0.055624,[2]
470439,312990,5,0,1,1.0,947,37243937,4.0,0.693147,6,0.142857,0.012675,0.02295,0.022691,[2]
179244,1511780,9,0,1,43.6,947,17079476,4.0,3.797734,1,0.500000,0.243055,0.44015,0.364747,[2]
372390,716490,14,0,1,50.4,947,30897590,5.0,3.939638,5,0.166667,0.105057,0.19025,0.174163,[2]
62270,1533420,19,0,1,30.4,947,7272922,5.0,3.446808,1,0.500000,0.275745,0.49935,0.405032,[2]
106173,809020,0,0,1,8.3,947,11373973,1.0,2.230014,4,0.200000,0.014272,0.02585,0.025522,[2]
165849,519860,25,0,1,19.4,947,16343859,5.0,3.015535,2,0.333333,0.160829,0.29124,0.255603,[2]
392998,1414850,9,7,1,0.1,1107,32094902,4.0,0.095310,2,0.333333,0.004067,0.00736,0.007333,[2]


In [12]:
#Number of games liked by each user, only includes users with at least one liked game
recommended_df = recommendations_short_data[recommendations_short_data['is_recommended'] == 1]

# Group by 'user_id' and calculate the sum of 'is_recommended' (True) for each user
user_recommended_counts = recommended_df.groupby('user_id').size().reset_index(name='recommendation_count')
user_recommended_counts

,user_id,recommendation_count
0,947,8
1,1107,4
2,1345,2
3,1405,9
4,1634,3
...,...,...
29364,14301844,10
29365,14303376,6
29366,14303661,10
29367,14305128,7


In [13]:
#Feed features for vectorizer, using description, and tags of games
def clean_data_list(x):
    return str(' '.join(x))

games_df['tags'] = games_df['tags'].apply(clean_data_list)
games_df['combined'] = games_df['description'] + ' ' + games_df['tags']
games_df.head(5)

,app_id,title,rating_encoded,positive_ratio_log,description,tags,combined
0,10,Counter-Strike,8.0,4.584967,Play the world's number 1 online action game. ...,Action FPS Multiplayer Shooter Classic Team-Ba...,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,7.0,4.454347,One of the most popular online action games of...,Action FPS Multiplayer Classic Hero Shooter Sh...,One of the most popular online action games of...
2,30,Day of Defeat,7.0,4.477337,Enlist in an intense brand of Axis vs. Allied ...,FPS World War II Multiplayer Shooter Action Wa...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Deathmatch Classic,5.0,4.382027,Enjoy fast-paced multiplayer gaming with Death...,Action FPS Classic Multiplayer Shooter First-P...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Half-Life: Opposing Force,8.0,4.564348,Return to the Black Mesa Research Facility as ...,FPS Action Classic Sci-fi Singleplayer Shooter...,Return to the Black Mesa Research Facility as ...


# Building Model

In [14]:
# Feature extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(games_df['combined'])

# Calculate Cosine Similarity
def cosine_similarity_n_space(m1, m2, batch_size=100):
    assert m1.shape[1] == m2.shape[1]
    ret = np.ndarray((m1.shape[0], m2.shape[0]))
    for row_i in range(0, int(m1.shape[0] / batch_size) + 1):
        start = row_i * batch_size
        end = min([(row_i + 1) * batch_size, m1.shape[0]])
        if end <= start:
            break # cause I'm too lazy to elegantly handle edge cases
        rows = m1[start: end]
        sim = cosine_similarity(rows, m2) # rows is O(1) size
        ret[start: end] = sim
    return ret

cosine_sim = cosine_similarity_n_space(tfidf_matrix, tfidf_matrix)

In [15]:
#Cosine Similarity Matrix
games_list = games_df['app_id'].tolist()
user_similarity_cosine_df = pd.DataFrame(cosine_sim)
user_similarity_cosine_df.index = games_list
user_similarity_cosine_df.columns = games_list


user_similarity_cosine_df

,10,20,30,40,50,60,70,80,130,220,...,2458580,2458940,2459460,2460050,2466740,2466750,2471820,2478270,2510770,2515240
10,1.000000,0.263759,0.165982,0.082715,0.063630,0.146549,0.086495,0.176870,0.052231,0.039310,...,0.015617,0.073046,0.030706,0.025012,0.016677,0.052603,0.021991,0.014286,0.000000,0.021557
20,0.263759,1.000000,0.188446,0.164895,0.073042,0.213836,0.085564,0.170044,0.073040,0.047832,...,0.021169,0.016923,0.040611,0.016624,0.030404,0.035694,0.017744,0.030565,0.007988,0.022925
30,0.165982,0.188446,1.000000,0.083456,0.084998,0.089816,0.074967,0.113878,0.043729,0.027484,...,0.015868,0.016056,0.040722,0.010306,0.006088,0.013335,0.007812,0.008419,0.001436,0.014471
40,0.082715,0.164895,0.083456,1.000000,0.089542,0.193020,0.181064,0.098331,0.103684,0.098447,...,0.000000,0.000000,0.045535,0.012077,0.005592,0.013796,0.007578,0.016592,0.009366,0.031313
50,0.063630,0.073042,0.084998,0.089542,1.000000,0.101303,0.225518,0.111499,0.336664,0.171834,...,0.013711,0.002329,0.026660,0.025983,0.029953,0.021783,0.033982,0.031661,0.012790,0.039568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466750,0.052603,0.035694,0.013335,0.013796,0.021783,0.057310,0.064515,0.018279,0.019437,0.013425,...,0.026293,0.032503,0.058007,0.013586,0.026581,1.000000,0.039165,0.018251,0.004575,0.014366
2471820,0.021991,0.017744,0.007812,0.007578,0.033982,0.056169,0.044185,0.078686,0.015970,0.028155,...,0.091801,0.000000,0.012250,0.130555,0.056922,0.039165,1.000000,0.037778,0.064411,0.111011
2478270,0.014286,0.030565,0.008419,0.016592,0.031661,0.039759,0.036784,0.018158,0.032432,0.033067,...,0.008338,0.036660,0.028540,0.034338,0.029171,0.018251,0.037778,1.000000,0.005482,0.011788
2510770,0.000000,0.007988,0.001436,0.009366,0.012790,0.000000,0.018734,0.006868,0.003441,0.003281,...,0.008301,0.002473,0.004967,0.008092,0.007250,0.004575,0.064411,0.005482,1.000000,0.026862


In [16]:
# Build the recommendation system

#If no liked games
def recommend_games_1(game_id, cosine_sim, df):
    # Get the index of the game
    game_index = df[df['app_id'] == game_id].index[0]
    # Get pairwise similarity scores
    sim_scores = list(enumerate(cosine_sim[game_index]))
    # Sort games based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get top N similar games
    top_similar_games = sim_scores[1:11]  # Exclude the first item which is the game itself
    # Get recommended game titles and similarity scores
    recommended_games = [{'title': df.iloc[sim_game[0]]['title'], 'cosine_sim': sim_game[1]} for sim_game in top_similar_games]
    # Create a DataFrame for recommended games
    recommended_df = pd.DataFrame(recommended_games)
    recommended_df = recommended_df.sort_values(by='cosine_sim', ascending=False)
    recommended_df.reset_index(drop=True, inplace=True)
    recommended_df.index = recommended_df.index + 1
    return recommended_df

#If have at least one liked game, calculates cosine similarity score of all games compared to users' liked games
def recommend_games_2(user_id, cosine_sim, df, top_n):
    # Getting games liked by user
    filtered_df = recommendations_short_data[(recommendations_short_data['user_id'] == user_id)
                                             & (recommendations_short_data['is_recommended'] == True)]
    app_ids = filtered_df['app_id'].tolist()
    recommended_games_list = []
    for app_id in app_ids:
        # Get the index of the game
        game_index = df[df['app_id'] == app_id].index[0]
        # Get pairwise similarity scores
        sim_scores = list(enumerate(cosine_sim[game_index]))
        # Sort games based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        # Get top N similar games
        top_similar_games = sim_scores[1:top_n+1]  # Exclude the first item which is the game itself
        # Get recommended game titles and similarity scores
        recommended_games = [{'title': df.iloc[sim_game[0]]['title'], 'cosine_sim': sim_game[1]} for sim_game in top_similar_games]
        # Append recommended games to list
        recommended_games_list.extend(recommended_games)

    recommended_df = pd.DataFrame(recommended_games_list)
    recommended_df = recommended_df.sort_values(by='cosine_sim', ascending=False)
    recommended_df = pd.merge(recommended_df, games_df[['title', 'app_id']], on='title', how='left')
    recommended_df = recommended_df[['app_id', 'title', 'cosine_sim']]
    recommended_df.reset_index(drop=True, inplace=True)
    recommended_df.index = recommended_df.index + 1
    return recommended_df.head(top_n)


In [17]:
# Recommend function, outputs recommendations in a DataFrame with app_id, 
# title, and cosine similiarty score to users' liked games

def recommend(user_id, top_n):
    # Checking if user has any liked games
    if user_id in user_recommended_counts['user_id'].values:
        filtered_df = recommendations_short_data[(recommendations_short_data['user_id'] == user_id) & (recommendations_short_data['is_recommended'] == True)]
        app_ids = filtered_df['app_id'].tolist()
        result = games_df[games_df['app_id'].isin(app_ids)]
        #Filter games out with 4.0 or less rating
        result = result[result['rating_encoded'] > 4.0]
        output = recommend_games_2(user_id, cosine_sim, games_df, top_n)
    else:
        game_id = games_df['app_id'].sample().values[0]
        #Filter games out with 4.0 or less rating
        filtered_games_df = games_df[games_df['rating_encoded'] > 4.0]
        output = recommend_games_1(game_id, cosine_sim, filtered_games_df)

    #print(result[['app_id', 'title']])
    return output


#recommended_games_df = recommend_games_1(game_title, cosine_sim, games_df)
#recommended_games_df.reset_index(drop=True, inplace=True)
#recommended_games_df.index = recommended_games_df.index + 1
#print(recommended_games_df)

In [18]:
# Output recommendations' app_ids in a list
def get_rec_list(user_id, k):
    try:
        recommendations = recommend(user_id, k)
        output = recommendations['app_id'].tolist()
    except Exception as e:
        output = "no data"

    return output

# Result

In [19]:
# Recommend example
recommend(14305218, 10)

,app_id,title,cosine_sim
1,1178830,Bright Memory: Infinite,0.843881
2,1111760,World of Guns: VR,0.698424
3,1299120,Mosaique Neko Waifus 2,0.665189
4,1504020,Mosaique Neko Waifus 4,0.597309
5,2165610,Mosaique Neko Waifus 5,0.580978
6,1309440,Sweet House Love,0.508471
7,1893800,Scuffy Game,0.491962
8,1592180,Shadownest,0.489835
9,841350,NALOGI 2,0.489793
10,293860,White Noise Online,0.459615


In [20]:
# Recommend list example
get_rec_list(14305218, 10)

[1178830,
 1111760,
 1299120,
 1504020,
 2165610,
 1309440,
 1893800,
 1592180,
 841350,
 293860]

# Evaluation


In [21]:
# Get user's cluster label
def get_user_cluster(picked_userid):
    user_cluster = cluster_data[cluster_data['user_id'] == picked_userid]['cluster_label'].values

    if len(user_cluster) == 0:
        # print(f"User {picked_userid} is not found in any cluster.")
        pass
    else:
        # print(f"User {picked_userid} belongs to cluster {user_cluster[0]}.")
        pass

    return user_cluster[0]

In [22]:
# Get games played by cluster, sorted by popularity in descending order
def get_top_n_popular_games_per_cluster():
    df_with_clusters_with_information = pd.merge(cluster_data, games_df, on='app_id', how='left')
    cluster_game_counts = df_with_clusters_with_information.groupby(['cluster_label', 'app_id', 'positive_ratio_log']).size().reset_index(name='User_Count')

    cluster_games_with_info = pd.merge(cluster_game_counts, games_df, on='app_id', how='left')

    cluster_game_counts_sorted = cluster_games_with_info.sort_values(by=['cluster_label', 'User_Count'], ascending=[True, False])
    top_n_popular_games_per_cluster = cluster_game_counts_sorted.groupby('cluster_label').head(100000)

    return top_n_popular_games_per_cluster

In [23]:
# Check if games recommended are reflected in cluster list
def check_against_cluster_list(picked_userid, k, user_cluster, top_n_popular_games_per_cluster, ranked_item_score_merged_dataset):
    user_clusters = [int(x) for x in user_cluster[1:-1].split(',')]

    for cluster in user_clusters:
        user_cluster_info = top_n_popular_games_per_cluster[top_n_popular_games_per_cluster.apply(lambda row: str(cluster) in row['cluster_label'], axis=1)]

        is_in_cluster = ranked_item_score_merged_dataset['app_id'].head(k).isin(user_cluster_info['app_id'])
        true_false_counts = is_in_cluster.value_counts()

        # print(is_in_cluster)
        # print("Accuracy result of checking against cluster: " + str(cluster))
        # print(true_false_counts)

        if len(is_in_cluster) > 0 :
            precision_k_picked_user = sum(is_in_cluster)/len(is_in_cluster)
        else:
            precision_k_picked_user = 0
        # print(precision_k_picked_user)

        return precision_k_picked_user

# Wrapper

In [24]:
def recommend_and_evaluate(picked_userid, k = 20):

    ranked_item_score_merged_dataset = recommend(picked_userid, k)
    # print(ranked_item_score_merged_dataset[['game_id', 'game_score', 'title']].head(k))

    user_cluster = get_user_cluster(picked_userid)

    top_n_popular_games_per_cluster = get_top_n_popular_games_per_cluster()

    precision_k_picked_user = check_against_cluster_list(picked_userid, k, user_cluster, top_n_popular_games_per_cluster, ranked_item_score_merged_dataset)

    print("Accuracy for user " + str(picked_userid) + ":")
    print(precision_k_picked_user)

    return precision_k_picked_user

In [25]:
recommend_and_evaluate(947)

Accuracy for user 947:
0.05


0.05

# Calculate Precision@k

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
random_users = [1350133,10954951,11339361,11896725,1584403,5206114,3340044,4912452,6232749,9369778,6031733,
 10877062,1978459,8080330,7660555,7458480,13647623,7307432,12460713,7661155,8754750,2488732,5158869,5017400,
 5559584,11039550,7717695,2286113,10439746,4004179]

In [28]:
N = 30
total = 0
k= 20

for user in random_users:
    picked_userid = user
    try:
        precision_k_for_picked_user = recommend_and_evaluate(picked_userid, k)
        total += precision_k_for_picked_user
    except Exception as e:
        continue

precision_k_n = total/N
print(f"Precision@K for N users, where K= {k} and N= {N}: ")
print(precision_k_n)

Accuracy for user 1350133:
0.5
Accuracy for user 10954951:
0.8
Accuracy for user 11339361:
0.3
Accuracy for user 11896725:
0.35
Accuracy for user 1584403:
0.5
Accuracy for user 5206114:
0.4
Accuracy for user 3340044:
0.75
Accuracy for user 4912452:
0.2
Accuracy for user 6232749:
0.45
Accuracy for user 9369778:
0.45
Accuracy for user 6031733:
0.4
Accuracy for user 10877062:
0.55
Accuracy for user 1978459:
0.25
Accuracy for user 8080330:
0.6
Accuracy for user 7660555:
0.55
Accuracy for user 7458480:
0.7
Accuracy for user 13647623:
0.35
Accuracy for user 7307432:
0.4
Accuracy for user 12460713:
0.5
Accuracy for user 7661155:
0.55
Accuracy for user 8754750:
0.35
Accuracy for user 2488732:
0.3
Accuracy for user 5158869:
0.6
Accuracy for user 5017400:
0.55
Accuracy for user 5559584:
0.95
Accuracy for user 11039550:
0.35
Accuracy for user 7717695:
0.4
Accuracy for user 2286113:
0.65
Accuracy for user 10439746:
0.65
Accuracy for user 4004179:
0.45
Precision@K for N users, where K= 20 and N= 30

# Calculate Novelty Score

In [29]:

# Create the popularity_score dictionary
popularity_score = {}

# Populate the popularity_score dictionary based on the sample data
for index, row in games_df.iterrows():
    popularity_score[row['app_id']] = row['rating_encoded']/8

pop_dict = popularity_score

In [30]:
def novelty_metric(rec_list, pop_dict):
    """
    Calculate the novelty metric for a list of recommended items.

    Parameters:
        rec_list (list): List of recommended items.
        pop_dict (dict): Dictionary containing popularity scores for items.

    Returns:
        float: Mean novelty score of the recommended items.
    """
    pop_sum = []  # List to store popularity scores of recommended items
    for item in rec_list:
        if item in pop_dict.keys():  # Check if the item exists in the popularity dictionary
            pop_sum.append(pop_dict[item])  # Add the popularity score of the item to the list
    return np.mean(pop_sum)  # Calculate and return the mean popularity score of recommended items


In [31]:
# Set the number of recommendations and the number of users
n_recommendations = 20
N = 30

# Initialize total novelty score
total_novelty_score = 0

# Iterate over each user
for user in random_users:

    # Generate recommended app_ID for the user
    rec_list = get_rec_list(user, n_recommendations)

    # Check if rec_list is empty
    if rec_list == 'no data':
        novelty_score = 0  # If there are no recommendations, novelty score is 0
    else:
        # Calculate novelty score for the recommendations
        novelty_score = novelty_metric(rec_list, pop_dict)

    # Print user ID, recommended item IDs, and novelty score
    print(user, rec_list, novelty_score)

    # Accumulate novelty score
    total_novelty_score += novelty_score

# Calculate average novelty score
average_novelty_score = total_novelty_score / N

# Print the average novelty score
print("The average novelty score for {N} users is: ", average_novelty_score)


1350133 [582890, 1386890, 849940, 1008580, 756600, 1649030, 611020, 2430370, 630, 1068350, 671610, 1008580, 28000, 1730020, 1800650, 242720, 1000500, 433350, 1092050, 215470] 0.65625
10954951 [955050, 1879330, 1576910, 9480, 1960270, 1889930, 242590, 9480, 1827180, 981600, 278080, 345180, 2208530, 887420, 1629200, 1938330, 206420, 55230, 239070, 247950] 0.73125
11339361 [963000, 1171320, 1559680, 1467870, 465170, 1515290, 1939640, 569380, 689910, 1330930, 1505910, 1074530, 1657600, 1875530, 1770660, 1598930, 864550, 1098760, 1571090, 2324650] 0.6875
11896725 [599900, 278930, 1250900, 1696200, 448160, 1446720, 440810, 2001670, 286100, 1434880, 1023150, 1137260, 616030, 454100, 1410070, 2236390, 495230, 1195530, 1047440, 412670] 0.69375
1584403 [1579380, 2092700, 1559390, 610370, 1383050, 7110, 1797940, 966470, 1809300, 992300, 527420, 1559390, 504000, 390100, 288060, 1875530, 339160, 1460490, 485360, 236930] 0.775
5206114 [17500, 923790, 473810, 1301520, 1386890, 849940, 980850, 1560110

# Calculate Diversity Score

In [32]:
# Import the function get_item_matrix from the item_matrix module

from item_matrix import get_item_matrix

# Call the get_item_matrix function to obtain the item similarity matrix
item_sim_matrix = get_item_matrix()


Index label '10' does not exist.
30000
531429
50872


In [33]:
# Calculate Diversity Score
def ils_metric(rec_list, item_sim_matrix):
    """
    Calculate the Intra-List Similarity (ILS) metric for a list of recommended items.

    Parameters:
        rec_list (list): List of recommended item IDs.
        item_sim_matrix (dict): Item similarity matrix containing pairwise similarity scores between items.

    Returns:
        float: Intra-List Similarity (ILS) score.
    """
    sim_temp = 0  # Initialize a temporary variable to store the similarity sum
    for i in range(0, len(rec_list)):
        for j in range(i + 1, len(rec_list)):
            # Check if item j is in the similarity matrix for item i
            if rec_list[j] in item_sim_matrix[rec_list[i]]:
                # If yes, add the similarity score to sim_temp
                sim_temp += item_sim_matrix[rec_list[i]][rec_list[j]]
    # Calculate the ILS score by subtracting the normalized similarity sum from 1
    return 1 - (sim_temp / (len(rec_list) * (len(rec_list) - 1)))


In [34]:
# Define the number of recommendations and the number of users
n_recommendations = 20
N = 30

# Initialize the total diversity score
total_diversity_score = 0

# Iterate over each user
for user in random_users:
    # Generate recommended item IDs for the user
    rec_list = get_rec_list(user, n_recommendations)

    # Check if rec_list is empty
    if rec_list == 'no data':
        diversity_score = 0  # If there are no recommendations, diversity score is 0
    else:
        # Calculate diversity score for the recommendations using the ILS metric
        diversity_score = ils_metric(rec_list, item_sim_matrix)

    # Print user ID, recommended item IDs, and diversity score
    print(user, rec_list, diversity_score)

    # Accumulate diversity score
    total_diversity_score += diversity_score

# Calculate average diversity score
average_diversity_score = total_diversity_score / N

# Print the average diversity score
print(f"The average diversity score for {N} users is: ", average_diversity_score)


1350133 [582890, 1386890, 849940, 1008580, 756600, 1649030, 611020, 2430370, 630, 1068350, 671610, 1008580, 28000, 1730020, 1800650, 242720, 1000500, 433350, 1092050, 215470] 0.8510990885239703
10954951 [955050, 1879330, 1576910, 9480, 1960270, 1889930, 242590, 9480, 1827180, 981600, 278080, 345180, 2208530, 887420, 1629200, 1938330, 206420, 55230, 239070, 247950] 0.9223966488648079
11339361 [963000, 1171320, 1559680, 1467870, 465170, 1515290, 1939640, 569380, 689910, 1330930, 1505910, 1074530, 1657600, 1875530, 1770660, 1598930, 864550, 1098760, 1571090, 2324650] 0.8785520037879617
11896725 [599900, 278930, 1250900, 1696200, 448160, 1446720, 440810, 2001670, 286100, 1434880, 1023150, 1137260, 616030, 454100, 1410070, 2236390, 495230, 1195530, 1047440, 412670] 0.9280702074311012
1584403 [1579380, 2092700, 1559390, 610370, 1383050, 7110, 1797940, 966470, 1809300, 992300, 527420, 1559390, 504000, 390100, 288060, 1875530, 339160, 1460490, 485360, 236930] 0.929066607162699
5206114 [17500, 